In [ ]:
# Delta Recurrent Neural Network (Delta-RNN) Framework
#
# This gives an implementation of the Delta-RNN framework given in Ororbia et al. 2017, arXiv:1703.08864 [cs.CL], 
# https://arxiv.org/abs/1703.08864 using Python and Tensorflow.
#
# This IPython Notebook provides an example of how to call the associated library of Python scripts.  
# Ororbia et al. should be consulted to make sure of the correct hyperparameter values.
#
# Stuart Hagler, 2017

In [ ]:
# Imports
import math
import sys

# Local Imports
sys.path.insert(0, 'python')
from delta_rnn import delta_rnn_graph
from read_data import read_data
from tokens import text_elements_to_tokens

In [ ]:
# Flags
usecase_flg = 2  # 1 for predicting letters
                 # 2 for predicting words with cutoff for infrequent words

In [ ]:
# Network-specific hyperparameters
c_size = 100                        # Dimension of the state vector
h_size = 100                        # Dimension of the hidden vector

# Training hyperparameters
num_training_unfoldings = 10        # Number of training unfoldings
    
# General network hyperparameters
word_frequency_cutoff = 50          # Cutoff for infrequent words for usecase_flg = 2

# General training hyperparameters
base_batch_size = 96                # Batch size across all towers
clip_norm = 1.25                    # Norm for gradient clipping
learning_decay = 1/2                # Multiplier to decay the learn rate when required
learning_rate = 0.05                # Initial learning rate
momentum = 0.9                      # Momentum for training
num_epochs = 50                     # Total number of epochs to run the algorithm
num_validation_unfoldings = 1000    # Number of validation unfoldings
optimization_frequency = 5          # Number of unfoldings before optimization step
summary_frequency = 500             # Summary information is displayed after training this many batches

# Cluster
cluster_spec = { "worker" : [ "localhost:2222" ] }
num_gpus = [ 1 ]                    # Number of GPUs available for each worker host (must agree with cluster_spec)

# Data file
filename = 'data/text8.zip'

In [ ]:
# Prepare training, validation, test data sets
num_towers = sum(num_gpus)
batch_size = base_batch_size // num_towers
raw_data = read_data(usecase_flg, filename)
data, dictionary, reverse_dictionary, vocabulary_size = text_elements_to_tokens(usecase_flg, raw_data, 
                                                                                word_frequency_cutoff)
training_size = math.floor((0.9/11)*len(raw_data)/num_towers)
validation_size = math.floor((0.1/11)*len(raw_data)/num_towers)
test_size = math.floor((0.1/11)*len(raw_data)/num_towers)
training_text = []
validation_text = []
test_text = []
for i in range(num_towers):
    training_text.append(data[i*training_size:(i+1)*training_size])
    validation_text.append(data[num_towers*training_size + i*validation_size: \
                                num_towers*training_size + (i+1)*validation_size])
    test_text.append(data[num_towers*(training_size + validation_size) + i*test_size: \
                          num_towers*(training_size + validation_size) + (i+1)*test_size])

In [ ]:
print('Vocabulary Size: %d' % vocabulary_size)

# Initialize graph
graph = delta_rnn_graph(cluster_spec, num_gpus, c_size, h_size, vocabulary_size, num_training_unfoldings, 
                        num_validation_unfoldings, batch_size, optimization_frequency, clip_norm, momentum)
    
# Train graph
graph.train(learning_rate, learning_decay, num_epochs, summary_frequency, training_text, validation_text)